### Flagging all similar data with different address

In [2]:
import pandas as pd
import numpy as np

# File paths
input_path = r"C:\HP - OIT - 2024\Lefel\Task 9 - Facility 2020-2023\fac_sc1.2_23.xlsx"
output_path = r"C:\HP - OIT - 2024\Lefel\Task 9 - Facility 2020-2023\fac_sc1.2_23_flagged.xlsx"

# Read Excel file
df = pd.read_excel(input_path)

# Add a row number to preserve the original order
df['row_order'] = range(1, len(df) + 1)

# Define columns to compare (excluding 'dups', 'Address', and row order)
columns_to_check = [col for col in df.columns if col not in ['dups', 'Address', 'row_order']]

# Initialize flag column
df['address_conflict_flag'] = False

# Helper function to compare values safely (handles NaN and strips strings)
def safe_equal(val1, val2):
    if pd.isna(val1) and pd.isna(val2):
        return True
    if isinstance(val1, str):
        val1 = val1.strip()
    if isinstance(val2, str):
        val2 = val2.strip()
    return val1 == val2

# Compare each row with the next one
for i in range(len(df) - 1):
    row1 = df.iloc[i]
    row2 = df.iloc[i + 1]

    # Check if all selected columns match
    match = all(safe_equal(row1[col], row2[col]) for col in columns_to_check)

    if match:
        addr1 = str(row1['Address']).strip() if not pd.isna(row1['Address']) else ''
        addr2 = str(row2['Address']).strip() if not pd.isna(row2['Address']) else ''
        if addr1 != addr2:
            df.at[i, 'address_conflict_flag'] = True
            df.at[i + 1, 'address_conflict_flag'] = True

# Save the result to a new Excel file
df.to_excel(output_path, index=False)


### Removing Duplicates by True and Address

In [3]:
import pandas as pd
import numpy as np

# File paths
input_path = r"C:\HP - OIT - 2024\Lefel\Task 9 - Facility 2020-2023\fac_sc1.2_23_flagged.xlsx"
output_path = r"C:\HP - OIT - 2024\Lefel\Task 9 - Facility 2020-2023\fac_sc1.2_23_cleaned.xlsx"

# Read Excel file
df = pd.read_excel(input_path)

# Define columns to use for grouping (excluding address and flag-related columns)
group_cols = [col for col in df.columns if col not in ['Address', 'dups', 'row_order', 'address_conflict_flag']]

# Initialize column to track which rows to keep
df['keep'] = True

# Focus only on rows flagged as address conflict
conflict_df = df[df['address_conflict_flag'] == True].copy()

# Create a group key based on all relevant columns (excluding address)
conflict_df['group_key'] = conflict_df[group_cols].astype(str).agg('|'.join, axis=1)

# Evaluate each group
rows_to_keep = []
for group_key, group in conflict_df.groupby('group_key'):
    # Initially mark all rows in this group for removal
    df.loc[group.index, 'keep'] = False

    # Check which rows have valid (non-empty) addresses
    group['has_address'] = group['Address'].apply(lambda x: isinstance(x, str) and x.strip() != '')

    if group['has_address'].sum() == 1:
        # If only one row has an address, keep that one
        idx_to_keep = group[group['has_address']].index[0]
        df.loc[idx_to_keep, 'keep'] = True

    elif group['has_address'].sum() > 1:
        # If multiple rows have addresses, keep the first one
        idx_to_keep = group[group['has_address']].index[0]
        df.loc[idx_to_keep, 'keep'] = True

# Final filtering
df_cleaned = df[df['keep']].drop(columns=['keep'])

# Save cleaned dataset to Excel
df_cleaned.to_excel(output_path, index=False)


### Sorting all duplicatied data 

In [1]:
import pandas as pd

# File paths for input and output
input_path = r"C:\HP - OIT - 2024\Lefel\Task 9 - Facility 2020-2023\fac_sc1.2_23_cleaned.xlsx"
output_path = r"C:\HP - OIT - 2024\Lefel\Task 9 - Facility 2020-2023\fac_sc1.2_23_same_facility_scope_flagged.xlsx"

# Read the cleaned dataset
df = pd.read_excel(input_path)

# Initialize flag column
df['same_facility_scope_flag'] = False

# Create a composite key from selected columns
df['combo_key'] = df[['acct.facility.actual.year.clean', 'facility_scope_1', 'facility_scope_2']].astype(str).agg('|'.join, axis=1)

# Find keys that appear more than once
duplicate_keys = df['combo_key'].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index

# Flag rows that have a duplicate key
df.loc[df['combo_key'].isin(duplicate_keys), 'same_facility_scope_flag'] = True

# Drop the temporary combo_key column
df.drop(columns=['combo_key'], inplace=True)

# Save the final output with flags
df.to_excel(output_path, index=False)


### Sorting with considering actual year and other columns

In [2]:
import pandas as pd

# File paths for input and output
input_path = r"C:\HP - OIT - 2024\Lefel\Task 9 - Facility 2020-2023\fac_sc1.2_23_cleaned.xlsx"
output_path = r"C:\HP - OIT - 2024\Lefel\Task 9 - Facility 2020-2023\fac_sc1.2_23_same_facility_scope_flagged2.xlsx"

# Read the cleaned dataset
df = pd.read_excel(input_path)

# Initialize the flag column
df['same_facility_scope_flag'] = False

# Create a composite key from four columns
df['combo_key'] = df[['acct.facility.actual.year.clean', 'facility_scope_1', 'facility_scope_2', 'actual_year']].astype(str).agg('|'.join, axis=1)

# Identify duplicate keys (i.e., repeated combinations)
duplicate_keys = df['combo_key'].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index

# Flag rows with duplicate keys
df.loc[df['combo_key'].isin(duplicate_keys), 'same_facility_scope_flag'] = True

# Drop the temporary composite key
df.drop(columns=['combo_key'], inplace=True)

# Save the output with flagged rows
df.to_excel(output_path, index=False)
